In [10]:
import cv2
import numpy as np
import glob
import pytesseract
import pandas as pd 

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [327]:
def rotateImage(images):
    for i, image in enumerate(images):
        img = Image.open(image)
        img = img.rotate(14)
        img.save('Datasets\\rotated\\plate' + str(i) + '.png', 'PNG')

def adaptiveThreshold(plates):
    for i, plate in enumerate(plates):
        img = cv2.imread(plate)

        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #cv2.imshow('gray', gray)

        ret, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
        # cv2.imshow('thresh', thresh)

        threshMean = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 10)
        # cv2.imshow('threshMean', threshMean)

        threshGauss = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 51, 27)
        #cv2.imshow('threshGauss', threshGauss)
        cv2.imwrite("Datasets\\processed\\plate{}.png".format(i), threshGauss)
        #cv2.waitKey(0)


def resize(processed):
    for i, image in enumerate(processed):
        image = cv2.imread(image)

        ratio = 600.0 / image.shape[1]
        dim = (600, int(image.shape[0] * ratio))

        resizedCubic = cv2.resize(image, dim, interpolation=cv2.INTER_CUBIC)

        cv2.imwrite("Datasets\\resized\\plate{}.png".format(i), resizedCubic)
        

def addBorder(resized):
    for i, image in enumerate(resized):
        image = cv2.imread(image)

        bordersize = 10
        border = cv2.copyMakeBorder(image, top=bordersize, bottom=bordersize, left=bordersize, right=bordersize,
                                    borderType=cv2.BORDER_CONSTANT, value=[255, 255, 255])

        cv2.imwrite("Datasets\\borders\\plate{}.png".format(i), border)
        

def cleanOCR(borders):
    detectedOCR = []

    for i, image in enumerate(borders):
        image = cv2.imread(image)

        edges = cv2.Canny(image, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(image=edges, rho=1, theta=np.pi / 180, threshold=100, lines=np.array([]),
                                minLineLength=100, maxLineGap=80)

        a, b, c = lines.shape
        for i in range(a):
            x = lines[i][0][0] - lines[i][0][2]
            y = lines[i][0][1] - lines[i][0][3]
            if x != 0:
                if abs(y / x) < 1:
                    cv2.line(image, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (255, 255, 255),
                             1, cv2.LINE_AA)

        se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
        gray = cv2.morphologyEx(image, cv2.MORPH_CLOSE, se)

        # OCR
        config = '-l eng --oem 1 --psm 3'
        text = pytesseract.image_to_string(gray, config=config)

        validChars = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
                      'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

        cleanText = []

        for char in text:
            if char in validChars:
                cleanText.append(char)

        plate = ''.join(cleanText)
        # print(plate)

        detectedOCR.append(plate)

        # cv2.imshow('img', gray)
        

    return detectedOCR

In [328]:
plates = glob.glob("Datasets\\plates\\*.png")
rotateImage(plates)

In [329]:
rotated_plates = glob.glob("Datasets\\rotated\\*.png")
adaptiveThreshold(rotated_plates)


In [330]:
processed = glob.glob("Datasets\\processed\\*.png")
resize(processed)


In [331]:
resized = glob.glob("Datasets\\resized\\*.png")
addBorder(resized)


In [332]:
bordered = glob.glob("Datasets\\borders\\*.png")
platesList = cleanOCR(bordered)

In [333]:
print(platesList)
plates = pd.DataFrame(platesList)
plates

['GRSUA70A', 'G', '', 'JER37T']


,0
0,GRSUA70A
1,G
2,
3,JER37T
